In [ ]:
# =====================
# Imports & Setup
# =====================
!pip install -q datasets sentencepiece sacrebleu torch torchvision torchaudio tqdm

import os, random
from pathlib import Path
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from datasets import load_dataset
import sentencepiece as spm
import sacrebleu

# =====================
# Config
# =====================
SEED = 42
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
VOCAB_SIZE = 16000
BATCH_SIZE = 64
MAX_LEN = 64
MAX_GEN_LEN = 64
EPOCHS = 30
CLIP = 1.0
LEARNING_RATE = 3e-4
BEAM_SIZE = 5
BEST_MODEL_PATH = Path("best_model_baseline.pt")
CHECKPOINT_PATH = Path("checkpoint_baseline.pt")
PATIENCE = 5  # early stopping patience

torch.manual_seed(SEED)
random.seed(SEED)

# =====================
# Load dataset (up to 1M)
# =====================
full_dataset = load_dataset("ai4bharat/samanantar", "hi", split="train")
full_dataset = full_dataset.shuffle(seed=SEED)

NUM_EXAMPLES = min(1_000_000, len(full_dataset))
subset = full_dataset.select(range(NUM_EXAMPLES))

# Split 80% train, 10% val, 10% test
train_end = int(0.8 * len(subset))
val_end = int(0.9 * len(subset))
train_data = subset.select(range(0, train_end))
val_data = subset.select(range(train_end, val_end))
test_data = subset.select(range(val_end, len(subset)))

print("Dataset sizes:", len(train_data), len(val_data), len(test_data))

# =====================
# SentencePiece
# =====================
SP_EN_MODEL = Path("spm_en.model")
SP_HI_MODEL = Path("spm_hi.model")

def write_lines(dataset_split, src_path, tgt_path):
    with open(src_path, "w", encoding="utf-8") as sf, open(tgt_path, "w", encoding="utf-8") as tf:
        for ex in dataset_split:
            sf.write(ex["src"].strip().lower() + "\n")
            tf.write(ex["tgt"].strip() + "\n")

write_lines(train_data, "train.en", "train.hi")

if not SP_EN_MODEL.exists() or not SP_HI_MODEL.exists():
    print("Training SentencePiece...")
    spm.SentencePieceTrainer.Train(
        f"--input=train.en --model_prefix=spm_en --vocab_size={VOCAB_SIZE} "
        f"--character_coverage=1.0 --model_type=unigram"
    )
    spm.SentencePieceTrainer.Train(
        f"--input=train.hi --model_prefix=spm_hi --vocab_size={VOCAB_SIZE} "
        f"--character_coverage=0.9995 --model_type=unigram"
    )

sp_en = spm.SentencePieceProcessor()
sp_hi = spm.SentencePieceProcessor()
sp_en.load(str(SP_EN_MODEL))
sp_hi.load(str(SP_HI_MODEL))

PAD_EN, BOS_EN, EOS_EN = 0, 1, 2
PAD_HI, BOS_HI, EOS_HI = 0, 1, 2

# =====================
# Dataset & DataLoader
# =====================
class NMTDataset(Dataset):
    def __init__(self, dataset, src_sp, tgt_sp, max_len=MAX_LEN):
        self.dataset = dataset
        self.src_sp = src_sp
        self.tgt_sp = tgt_sp
        self.max_len = max_len

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        src_text = self.dataset[idx]["src"].lower()
        tgt_text = self.dataset[idx]["tgt"]
        src_ids = [BOS_EN] + self.src_sp.encode(src_text)[:self.max_len-2] + [EOS_EN]
        tgt_ids = [BOS_HI] + self.tgt_sp.encode(tgt_text)[:self.max_len-2] + [EOS_HI]
        src_ids += [PAD_EN] * (self.max_len - len(src_ids))
        tgt_ids += [PAD_HI] * (self.max_len - len(tgt_ids))
        return torch.tensor(src_ids), torch.tensor(tgt_ids)

def get_loader(dataset_split, shuffle=True):
    return DataLoader(NMTDataset(dataset_split, sp_en, sp_hi),
                      batch_size=BATCH_SIZE, shuffle=shuffle)

train_loader = get_loader(train_data)
val_loader = get_loader(val_data)
test_loader = get_loader(test_data, shuffle=False)

# =====================
# Masks
# =====================
def create_padding_mask(seq, lang='en'):
    pad_id = PAD_EN if lang == 'en' else PAD_HI
    return (seq == pad_id)

def generate_square_subsequent_mask(sz):
    return torch.triu(torch.ones(sz, sz, device=DEVICE) * float('-inf'), diagonal=1)

# =====================
# Transformer Model
# =====================
class TransformerModel(nn.Module):
    def __init__(self, src_vocab, tgt_vocab, d_model=512, nhead=8,
                 num_layers=3, dim_feedforward=1024, dropout=0.1):
        super().__init__()
        self.src_emb = nn.Embedding(src_vocab, d_model, padding_idx=PAD_EN)
        self.tgt_emb = nn.Embedding(tgt_vocab, d_model, padding_idx=PAD_HI)
        self.pos_enc = nn.Parameter(torch.zeros(1, MAX_LEN, d_model))
        self.transformer = nn.Transformer(
            d_model=d_model, nhead=nhead,
            num_encoder_layers=num_layers, num_decoder_layers=num_layers,
            dim_feedforward=dim_feedforward, dropout=dropout,
            batch_first=True
        )
        self.fc_out = nn.Linear(d_model, tgt_vocab)

    def encode(self, src, src_key_padding_mask):
        src_emb = self.src_emb(src) + self.pos_enc[:, :src.size(1), :]
        return self.transformer.encoder(src_emb, src_key_padding_mask=src_key_padding_mask)

    def decode(self, tgt, memory, tgt_mask, memory_key_padding_mask, tgt_key_padding_mask):
        tgt_emb = self.tgt_emb(tgt) + self.pos_enc[:, :tgt.size(1), :]
        return self.transformer.decoder(
            tgt_emb, memory,
            tgt_mask=tgt_mask,
            memory_key_padding_mask=memory_key_padding_mask,
            tgt_key_padding_mask=tgt_key_padding_mask
        )

    def forward(self, src, tgt, src_key_padding_mask=None, tgt_key_padding_mask=None,
                memory_key_padding_mask=None, tgt_mask=None):
        memory = self.encode(src, src_key_padding_mask)
        output = self.decode(tgt, memory, tgt_mask, memory_key_padding_mask, tgt_key_padding_mask)
        return self.fc_out(output)

# =====================
# Training Setup
# =====================
model = TransformerModel(len(sp_en), len(sp_hi)).to(DEVICE)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_HI)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=LEARNING_RATE, steps_per_epoch=len(train_loader), epochs=EPOCHS
)
scaler = torch.amp.GradScaler()

# ---- Resume if checkpoint exists ----
start_epoch = 1
best_val_loss = float("inf")
epochs_no_improve = 0

if CHECKPOINT_PATH.exists():
    print(f"🔄 Resuming from checkpoint: {CHECKPOINT_PATH}")
    checkpoint = torch.load(CHECKPOINT_PATH, map_location=DEVICE)
    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    scheduler.load_state_dict(checkpoint["scheduler_state_dict"])
    # Handle optional scaler
    if "scaler_state_dict" in checkpoint:
        scaler.load_state_dict(checkpoint["scaler_state_dict"])
        print("✅ Loaded scaler state from checkpoint.")
    else:
        print("⚠️ No scaler state found in checkpoint — continuing without it.")
    start_epoch = checkpoint.get("epoch", 0) + 1
    best_val_loss = checkpoint.get("best_val_loss", float("inf"))
    epochs_no_improve = checkpoint.get("epochs_no_improve", 0)
    print(f"Resumed from epoch {start_epoch-1}, best val loss = {best_val_loss:.4f}")

# =====================
# Training Loop
# =====================
for epoch in range(start_epoch, EPOCHS + 1):
    # ---- Train ----
    model.train()
    train_loss = 0
    for src, tgt in tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}", unit="batch"):
        src, tgt = src.to(DEVICE), tgt.to(DEVICE)
        src_mask = create_padding_mask(src, 'en')
        tgt_input, tgt_out = tgt[:, :-1], tgt[:, 1:]
        tgt_mask = generate_square_subsequent_mask(tgt_input.size(1))
        tgt_key_padding_mask = create_padding_mask(tgt_input, 'hi')

        optimizer.zero_grad()
        with torch.amp.autocast(device_type="cuda"):
            logits = model(
                src, tgt_input,
                src_key_padding_mask=src_mask,
                tgt_key_padding_mask=tgt_key_padding_mask,
                memory_key_padding_mask=src_mask,
                tgt_mask=tgt_mask
            )
            loss = criterion(logits.reshape(-1, logits.size(-1)), tgt_out.reshape(-1))

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP)
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)

    # ---- Validation ----
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for src, tgt in val_loader:
            src, tgt = src.to(DEVICE), tgt.to(DEVICE)
            src_mask = create_padding_mask(src, 'en')
            tgt_input, tgt_out = tgt[:, :-1], tgt[:, 1:]
            tgt_mask = generate_square_subsequent_mask(tgt_input.size(1))
            tgt_key_padding_mask = create_padding_mask(tgt_input, 'hi')

            logits = model(
                src, tgt_input,
                src_key_padding_mask=src_mask,
                tgt_key_padding_mask=tgt_key_padding_mask,
                memory_key_padding_mask=src_mask,
                tgt_mask=tgt_mask
            )
            loss = criterion(logits.reshape(-1, logits.size(-1)), tgt_out.reshape(-1))
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"Epoch {epoch} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

    # ---- Save Checkpoint ----
    torch.save({
        "epoch": epoch,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "scheduler_state_dict": scheduler.state_dict(),
        "scaler_state_dict": scaler.state_dict(),
        "best_val_loss": best_val_loss,
        "epochs_no_improve": epochs_no_improve,
    }, CHECKPOINT_PATH)

    # ---- Early Stopping ----
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        print("✅ New best model saved!")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= PATIENCE:
            print("⏹️ Early stopping triggered.")
            break

print("🎯 Training complete.")


C:\Users\Ayush\.conda\envs\mt_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset sizes: 800000 100000 100000
🔄 Resuming from checkpoint: checkpoint_baseline.pt


C:\Users\Ayush\AppData\Local\Temp\ipykernel_28892\798033649.py:183: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(CHECKPOINT_PATH, map_location=DEVIC

✅ Loaded scaler state from checkpoint.
Resumed from epoch 26, best val loss = 2.4229


Epoch 27/30:   0%|                                                                        | 0/12500 [00:00<?, ?batch/s]C:\Users\Ayush\.conda\envs\mt_env\lib\site-packages\torch\nn\functional.py:5849: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
Epoch 27/30: 100%|████████████████████████████████████████████████████████████| 12500/12500 [13:56<00:00, 14.94batch/s]
C:\Users\Ayush\.conda\envs\mt_env\lib\site-packages\torch\nn\modules\transformer.py:502: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\NestedTensorImpl.cpp:180.)
  output = torch._nested_tensor_from_mask(


Epoch 27 | Train Loss: 1.5976 | Val Loss: 2.4865


Epoch 28/30: 100%|████████████████████████████████████████████████████████████| 12500/12500 [13:56<00:00, 14.93batch/s]


Epoch 28 | Train Loss: 1.5685 | Val Loss: 2.4935


Epoch 29/30: 100%|████████████████████████████████████████████████████████████| 12500/12500 [13:54<00:00, 14.98batch/s]


Epoch 29 | Train Loss: 1.5502 | Val Loss: 2.4999
⏹️ Early stopping triggered.
🎯 Training complete.


In [ ]:
# =====================
# Save Full Checkpoint
# =====================
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'best_val_loss': best_val_loss,
    'config': {
        'VOCAB_SIZE': VOCAB_SIZE,
        'MAX_LEN': MAX_LEN,
        'EPOCHS': EPOCHS,
        'LEARNING_RATE': LEARNING_RATE,
        'CLIP': CLIP,
        'BATCH_SIZE': BATCH_SIZE
    }
}, "checkpoint_baseline.pt")

print("✅ Full checkpoint saved as 'checkpoint_baseline.pt'")


✅ Full checkpoint saved as 'checkpoint_baseline.pt'


In [ ]:
import json

config = {
    "VOCAB_SIZE": VOCAB_SIZE,
    "MAX_LEN": MAX_LEN,
    "EPOCHS": EPOCHS,
    "LEARNING_RATE": LEARNING_RATE,
    "CLIP": CLIP,
    "BATCH_SIZE": BATCH_SIZE,
    "DEVICE": DEVICE
}

with open("config.json", "w") as f:
    json.dump(config, f, indent=4)

print("✅ Config saved to config.json")


✅ Config saved to config.json


In [ ]:
# =====================
# ✅ Evaluation on Test Set
# =====================
from torch.nn.functional import log_softmax
import math

# ---- Load best model ----
model.load_state_dict(torch.load(BEST_MODEL_PATH, map_location=DEVICE))
model.eval()
print("✅ Loaded best model for evaluation.")

# =====================
# Greedy Decoding
# =====================
def greedy_decode(model, src, max_len=MAX_GEN_LEN):
    model.eval()
    src_mask = create_padding_mask(src, 'en')
    memory = model.encode(src, src_mask)
    ys = torch.full((src.size(0), 1), BOS_HI, dtype=torch.long, device=DEVICE)

    for i in range(max_len - 1):
        tgt_mask = generate_square_subsequent_mask(ys.size(1))
        tgt_key_padding_mask = create_padding_mask(ys, 'hi')

        out = model.decode(
            ys, memory, tgt_mask=tgt_mask,
            memory_key_padding_mask=src_mask,
            tgt_key_padding_mask=tgt_key_padding_mask
        )
        out = model.fc_out(out[:, -1, :])
        next_word = out.argmax(dim=-1, keepdim=True)
        ys = torch.cat([ys, next_word], dim=1)

        # Stop if EOS is reached for all
        if torch.all(next_word.squeeze() == EOS_HI):
            break
    return ys


# =====================
# Beam Search Decoding
# =====================
def beam_search_decode(model, src, beam_size=BEAM_SIZE, max_len=MAX_GEN_LEN):
    model.eval()
    src_mask = create_padding_mask(src, 'en')
    memory = model.encode(src, src_mask)

    batch_size = src.size(0)
    results = []

    for i in range(batch_size):
        beams = [(torch.tensor([[BOS_HI]], device=DEVICE), 0.0)]
        for _ in range(max_len - 1):
            new_beams = []
            for seq, score in beams:
                if seq[0, -1].item() == EOS_HI:
                    new_beams.append((seq, score))
                    continue

                tgt_mask = generate_square_subsequent_mask(seq.size(1))
                tgt_key_padding_mask = create_padding_mask(seq, 'hi')
                out = model.decode(
                    seq, memory[i:i+1],
                    tgt_mask=tgt_mask,
                    memory_key_padding_mask=src_mask[i:i+1],
                    tgt_key_padding_mask=tgt_key_padding_mask
                )
                logits = model.fc_out(out[:, -1, :])
                log_probs = log_softmax(logits, dim=-1)
                topk_log_probs, topk_indices = torch.topk(log_probs, beam_size)

                for k in range(beam_size):
                    next_seq = torch.cat([seq, topk_indices[:, k].unsqueeze(1)], dim=1)
                    new_beams.append((next_seq, score + topk_log_probs[0, k].item()))

            beams = sorted(new_beams, key=lambda x: x[1], reverse=True)[:beam_size]

            if all(seq[0, -1].item() == EOS_HI for seq, _ in beams):
                break

        best_seq = beams[0][0]
        results.append(best_seq)
    return results


# =====================
# Evaluate BLEU on Test Set
# =====================
references, hypotheses = [], []
for batch_idx, (src, tgt) in enumerate(tqdm(test_loader, desc="Evaluating", unit="batch")):
    src, tgt = src.to(DEVICE), tgt.to(DEVICE)
    with torch.no_grad():
        pred = greedy_decode(model, src)
    for i in range(src.size(0)):
        tgt_tokens = tgt[i].tolist()
        pred_tokens = pred[i].tolist()

        tgt_text = sp_hi.decode([t for t in tgt_tokens if t not in [PAD_HI, BOS_HI, EOS_HI]])
        pred_text = sp_hi.decode([t for t in pred_tokens if t not in [PAD_HI, BOS_HI, EOS_HI]])

        references.append([tgt_text])
        hypotheses.append(pred_text)

# ---- Compute BLEU ----
bleu = sacrebleu.corpus_bleu(hypotheses, list(zip(*references)))
print(f"🌍 Test BLEU score: {bleu.score:.2f}")

# =====================
# Show Some Examples
# =====================
print("\n🔍 Sample Translations:")
for i in range(5):
    src_ids, tgt_ids = test_data[i]["src"], test_data[i]["tgt"]
    src_tensor = torch.tensor([[BOS_EN] + sp_en.encode(src_ids.lower())[:MAX_LEN-2] + [EOS_EN]], device=DEVICE)
    pred = greedy_decode(model, src_tensor)
    pred_tokens = pred[0].tolist()
    pred_text = sp_hi.decode([t for t in pred_tokens if t not in [PAD_HI, BOS_HI, EOS_HI]])

    print(f"\nEN: {test_data[i]['src']}")
    print(f"HI (Reference): {test_data[i]['tgt']}")
    print(f"HI (Predicted): {pred_text}")
    print("-" * 50)


C:\Users\Ayush\AppData\Local\Temp\ipykernel_28892\2853380784.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(BEST_MODEL_PATH, map_locati

✅ Loaded best model for evaluation.


Evaluating: 100%|███████████████████████████████████████████████████████████████| 1563/1563 [09:34<00:00,  2.72batch/s]


🌍 Test BLEU score: 8.58

🔍 Sample Translations:

EN: Its movement was captured on CCTV.
HI (Reference): उनकी यह हरकत सीसीटीवी में कैद हो गई।
HI (Predicted): यह सीसीटीवी में कैद हो गया।
--------------------------------------------------

EN: "The two leaders ""discussed bilateral ties, including development partnership and cooperation in counter-terrorism and international fora,"" he said in the tweet."
HI (Reference): ट्वीट में उन्होंने कहा कि दोनों नेताओं ने ‘‘विकास साझेदारी और आतंकवाद के खिलाफ तथा अंतरराष्ट्रीय मंचों पर सहयोग समेत द्विपक्षीय संबंधों’’ पर चर्चा की।
HI (Predicted): उन्होंने ट्वीट किया, ‘‘दोनों नेताओं ने द्विपक्षीय संबंधों को तोड़ा, जिसमें विकास साझेदारी और आतंकवाद विरोधी और अंतर्राष्‍ट्रीय मंचों पर सहयोग शामिल हैं।
--------------------------------------------------

EN: These are Allahabad Bank, United Bank of India, Corporation Bank, IDBI Bank, Uco Bank, Bank of India, Central Bank of India, Indian Overseas Bank, Oriental Bank of Commerce, Dena Bank and Bank of Mahara

In [ ]:
# =====================
# Imports & Setup
# =====================
!pip install -q datasets sentencepiece sacrebleu torch torchvision torchaudio tqdm

import os, random
from pathlib import Path
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from datasets import load_dataset
import sentencepiece as spm
import sacrebleu

# =====================
# Config
# =====================
SEED = 42
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
VOCAB_SIZE = 16000
BATCH_SIZE = 64
MAX_LEN = 64
MAX_GEN_LEN = 64
EPOCHS = 30
CLIP = 1.0
LEARNING_RATE = 3e-4
BEAM_SIZE = 5
BEST_MODEL_PATH = Path("best_model_cnn.pt")
CHECKPOINT_PATH = Path("checkpoint_cnn.pt")
PATIENCE = 5

torch.manual_seed(SEED)
random.seed(SEED)

# =====================
# Load dataset (up to 1M)
# =====================
full_dataset = load_dataset("ai4bharat/samanantar", "hi", split="train")
full_dataset = full_dataset.shuffle(seed=SEED)

NUM_EXAMPLES = min(1_000_000, len(full_dataset))
subset = full_dataset.select(range(NUM_EXAMPLES))

# Split 80/10/10
train_end = int(0.8 * len(subset))
val_end = int(0.9 * len(subset))
train_data = subset.select(range(0, train_end))
val_data = subset.select(range(train_end, val_end))
test_data = subset.select(range(val_end, len(subset)))

print("Dataset sizes:", len(train_data), len(val_data), len(test_data))

# =====================
# SentencePiece
# =====================
SP_EN_MODEL = Path("spm_en.model")
SP_HI_MODEL = Path("spm_hi.model")

def write_lines(dataset_split, src_path, tgt_path):
    with open(src_path, "w", encoding="utf-8") as sf, open(tgt_path, "w", encoding="utf-8") as tf:
        for ex in dataset_split:
            sf.write(ex["src"].strip().lower() + "\n")
            tf.write(ex["tgt"].strip() + "\n")

write_lines(train_data, "train.en", "train.hi")

if not SP_EN_MODEL.exists() or not SP_HI_MODEL.exists():
    print("Training SentencePiece...")
    spm.SentencePieceTrainer.Train(
        f"--input=train.en --model_prefix=spm_en --vocab_size={VOCAB_SIZE} "
        f"--character_coverage=1.0 --model_type=unigram"
    )
    spm.SentencePieceTrainer.Train(
        f"--input=train.hi --model_prefix=spm_hi --vocab_size={VOCAB_SIZE} "
        f"--character_coverage=0.9995 --model_type=unigram"
    )

sp_en = spm.SentencePieceProcessor()
sp_hi = spm.SentencePieceProcessor()
sp_en.load(str(SP_EN_MODEL))
sp_hi.load(str(SP_HI_MODEL))

PAD_EN, BOS_EN, EOS_EN = 0, 1, 2
PAD_HI, BOS_HI, EOS_HI = 0, 1, 2

# =====================
# Dataset & DataLoader
# =====================
class NMTDataset(Dataset):
    def __init__(self, dataset, src_sp, tgt_sp, max_len=MAX_LEN):
        self.dataset = dataset
        self.src_sp = src_sp
        self.tgt_sp = tgt_sp
        self.max_len = max_len

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        src_text = self.dataset[idx]["src"].lower()
        tgt_text = self.dataset[idx]["tgt"]
        src_ids = [BOS_EN] + self.src_sp.encode(src_text)[:self.max_len-2] + [EOS_EN]
        tgt_ids = [BOS_HI] + self.tgt_sp.encode(tgt_text)[:self.max_len-2] + [EOS_HI]
        src_ids += [PAD_EN] * (self.max_len - len(src_ids))
        tgt_ids += [PAD_HI] * (self.max_len - len(tgt_ids))
        return torch.tensor(src_ids), torch.tensor(tgt_ids)

def get_loader(dataset_split, shuffle=True):
    return DataLoader(NMTDataset(dataset_split, sp_en, sp_hi),
                      batch_size=BATCH_SIZE, shuffle=shuffle)

train_loader = get_loader(train_data)
val_loader = get_loader(val_data)
test_loader = get_loader(test_data, shuffle=False)

# =====================
# Masks
# =====================
def create_padding_mask(seq, lang='en'):
    pad_id = PAD_EN if lang == 'en' else PAD_HI
    return (seq == pad_id)

def generate_square_subsequent_mask(sz):
    return torch.triu(torch.ones(sz, sz, device=DEVICE) * float('-inf'), diagonal=1)

# =====================
# CNN Feature Extractor (2-layer)
# =====================
class CNNFeatureExtractor(nn.Module):
    def __init__(self, embed_dim):
        super().__init__()
        self.conv1 = nn.Conv1d(embed_dim, embed_dim, kernel_size=5, padding=2)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(embed_dim, embed_dim, kernel_size=3, padding=1)
        self.norm = nn.LayerNorm(embed_dim)

    def forward(self, x):
        residual = x
        x = x.transpose(1, 2)
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = x.transpose(1, 2)
        return self.norm(x + residual)

# =====================
# Hybrid CNN + Transformer Model
# =====================
class HybridTransformerModel(nn.Module):
    def __init__(self, src_vocab, tgt_vocab, d_model=512, nhead=8,
                 num_layers=3, dim_feedforward=1024, dropout=0.1):
        super().__init__()
        self.src_emb = nn.Embedding(src_vocab, d_model, padding_idx=PAD_EN)
        self.tgt_emb = nn.Embedding(tgt_vocab, d_model, padding_idx=PAD_HI)
        self.pos_enc = nn.Parameter(torch.zeros(1, MAX_LEN, d_model))
        self.cnn_encoder = CNNFeatureExtractor(d_model)
        self.transformer = nn.Transformer(
            d_model=d_model, nhead=nhead,
            num_encoder_layers=num_layers, num_decoder_layers=num_layers,
            dim_feedforward=dim_feedforward, dropout=dropout, batch_first=True
        )
        self.fc_out = nn.Linear(d_model, tgt_vocab)

    def encode(self, src, src_key_padding_mask):
        src_emb = self.src_emb(src) + self.pos_enc[:, :src.size(1), :]
        src_cnn = self.cnn_encoder(src_emb)
        return self.transformer.encoder(src_cnn, src_key_padding_mask=src_key_padding_mask)

    def decode(self, tgt, memory, tgt_mask, memory_key_padding_mask, tgt_key_padding_mask):
        tgt_emb = self.tgt_emb(tgt) + self.pos_enc[:, :tgt.size(1), :]
        return self.transformer.decoder(
            tgt_emb, memory,
            tgt_mask=tgt_mask,
            memory_key_padding_mask=memory_key_padding_mask,
            tgt_key_padding_mask=tgt_key_padding_mask
        )

    def forward(self, src, tgt, src_key_padding_mask=None, tgt_key_padding_mask=None,
                memory_key_padding_mask=None, tgt_mask=None):
        memory = self.encode(src, src_key_padding_mask)
        output = self.decode(tgt, memory, tgt_mask, memory_key_padding_mask, tgt_key_padding_mask)
        return self.fc_out(output)

# =====================
# Training Setup
# =====================
model = HybridTransformerModel(len(sp_en), len(sp_hi)).to(DEVICE)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_HI)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=LEARNING_RATE, steps_per_epoch=len(train_loader), epochs=EPOCHS
)
scaler = torch.amp.GradScaler()

start_epoch = 1
best_val_loss = float("inf")
epochs_no_improve = 0

# =====================
# Resume from Checkpoint (if available)
# =====================
if CHECKPOINT_PATH.exists():
    print(f"🔁 Resuming from {CHECKPOINT_PATH}...")
    checkpoint = torch.load(CHECKPOINT_PATH, map_location=DEVICE)

    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    scaler.load_state_dict(checkpoint['scaler_state_dict'])

    start_epoch = checkpoint['epoch'] + 1
    best_val_loss = checkpoint['best_val_loss']
    epochs_no_improve = checkpoint['epochs_no_improve']

    print(f"✅ Checkpoint loaded — Resuming from epoch {start_epoch}")
else:
    print("🚀 Starting training from scratch")

# =====================
# Training Loop
# =====================
for epoch in range(start_epoch, EPOCHS + 1):
    model.train()
    train_loss = 0
    for src, tgt in tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}", unit="batch"):
        src, tgt = src.to(DEVICE), tgt.to(DEVICE)
        src_mask = create_padding_mask(src, 'en')
        tgt_input, tgt_out = tgt[:, :-1], tgt[:, 1:]
        tgt_mask = generate_square_subsequent_mask(tgt_input.size(1))
        tgt_key_padding_mask = create_padding_mask(tgt_input, 'hi')

        optimizer.zero_grad()
        with torch.amp.autocast(device_type="cuda"):
            logits = model(
                src, tgt_input,
                src_key_padding_mask=src_mask,
                tgt_key_padding_mask=tgt_key_padding_mask,
                memory_key_padding_mask=src_mask,
                tgt_mask=tgt_mask
            )
            loss = criterion(logits.reshape(-1, logits.size(-1)), tgt_out.reshape(-1))

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP)
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)

    # ---- Validation ----
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for src, tgt in val_loader:
            src, tgt = src.to(DEVICE), tgt.to(DEVICE)
            src_mask = create_padding_mask(src, 'en')
            tgt_input, tgt_out = tgt[:, :-1], tgt[:, 1:]
            tgt_mask = generate_square_subsequent_mask(tgt_input.size(1))
            tgt_key_padding_mask = create_padding_mask(tgt_input, 'hi')

            logits = model(
                src, tgt_input,
                src_key_padding_mask=src_mask,
                tgt_key_padding_mask=tgt_key_padding_mask,
                memory_key_padding_mask=src_mask,
                tgt_mask=tgt_mask
            )
            loss = criterion(logits.reshape(-1, logits.size(-1)), tgt_out.reshape(-1))
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"Epoch {epoch} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

    # ---- Save Checkpoint ----
    torch.save({
        "epoch": epoch,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "scheduler_state_dict": scheduler.state_dict(),
        "scaler_state_dict": scaler.state_dict(),
        "best_val_loss": best_val_loss,
        "epochs_no_improve": epochs_no_improve,
        "config": {
            "MODEL_TYPE": "HybridCNNTransformer",
            "VOCAB_SIZE": VOCAB_SIZE,
            "MAX_LEN": MAX_LEN,
            "EPOCHS": EPOCHS,
            "LEARNING_RATE": LEARNING_RATE,
            "CLIP": CLIP,
            "BATCH_SIZE": BATCH_SIZE
        }
    }, CHECKPOINT_PATH)

    # ---- Early Stopping ----
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        print("✅ New best model saved!")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= PATIENCE:
            print("⏹️ Early stopping triggered.")
            break

print("🎯 Training complete.")


C:\Users\Ayush\.conda\envs\mt_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset sizes: 800000 100000 100000
🔁 Resuming from checkpoint_cnn.pt...


C:\Users\Ayush\AppData\Local\Temp\ipykernel_13800\227662617.py:206: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(CHECKPOINT_PATH, map_location=DEVIC

✅ Checkpoint loaded — Resuming from epoch 23


Epoch 23/30:   0%|                                                                        | 0/12500 [00:00<?, ?batch/s]C:\Users\Ayush\.conda\envs\mt_env\lib\site-packages\torch\nn\functional.py:5849: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
Epoch 23/30: 100%|████████████████████████████████████████████████████████████| 12500/12500 [12:50<00:00, 16.22batch/s]
C:\Users\Ayush\.conda\envs\mt_env\lib\site-packages\torch\nn\modules\transformer.py:502: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\NestedTensorImpl.cpp:180.)
  output = torch._nested_tensor_from_mask(


Epoch 23 | Train Loss: 1.5992 | Val Loss: 2.4764


Epoch 24/30: 100%|████████████████████████████████████████████████████████████| 12500/12500 [12:51<00:00, 16.21batch/s]


Epoch 24 | Train Loss: 1.5465 | Val Loss: 2.4910
⏹️ Early stopping triggered.
🎯 Training complete.


In [ ]:
# =====================
# BLEU Evaluation + Translation
# =====================
import torch
import sacrebleu
from tqdm import tqdm
import random

# Load SentencePiece models again (if running separately)
sp_en = spm.SentencePieceProcessor()
sp_hi = spm.SentencePieceProcessor()
sp_en.load("spm_en.model")
sp_hi.load("spm_hi.model")

PAD_EN, BOS_EN, EOS_EN = 0, 1, 2
PAD_HI, BOS_HI, EOS_HI = 0, 1, 2

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Reload the best model
model = HybridTransformerModel(len(sp_en), len(sp_hi)).to(DEVICE)
model.load_state_dict(torch.load("best_model_cnn.pt", map_location=DEVICE))
model.eval()

# =====================
# Greedy / Beam Decoding
# =====================
@torch.no_grad()
def translate_sentence(sentence, model, sp_en, sp_hi, max_len=64, beam_size=5):
    model.eval()
    src_ids = [BOS_EN] + sp_en.encode(sentence.lower())[:max_len-2] + [EOS_EN]
    src = torch.tensor(src_ids, dtype=torch.long, device=DEVICE).unsqueeze(0)
    src_mask = create_padding_mask(src, 'en')

    memory = model.encode(src, src_mask)

    # ---- Start with BOS token ----
    tgt = torch.tensor([[BOS_HI]], dtype=torch.long, device=DEVICE)

    for _ in range(max_len):
        tgt_mask = generate_square_subsequent_mask(tgt.size(1))
        tgt_key_padding_mask = create_padding_mask(tgt, 'hi')

        output = model.decode(
            tgt, memory,
            tgt_mask=tgt_mask,
            memory_key_padding_mask=src_mask,
            tgt_key_padding_mask=tgt_key_padding_mask
        )
        logits = model.fc_out(output[:, -1, :])
        next_token = logits.argmax(-1).item()

        tgt = torch.cat([tgt, torch.tensor([[next_token]], device=DEVICE)], dim=1)
        if next_token == EOS_HI:
            break

    decoded = sp_hi.decode([t for t in tgt.squeeze().tolist() if t not in [BOS_HI, EOS_HI, PAD_HI]])
    return decoded


# =====================
# BLEU Evaluation
# =====================
refs, hyps = [], []

print("🔍 Evaluating BLEU on test set...")
for src, tgt in tqdm(test_loader, desc="Evaluating", unit="batch"):
    src, tgt = src.to(DEVICE), tgt.to(DEVICE)
    for i in range(src.size(0)):
        src_text = sp_en.decode([t for t in src[i].tolist() if t not in [BOS_EN, EOS_EN, PAD_EN]])
        tgt_text = sp_hi.decode([t for t in tgt[i].tolist() if t not in [BOS_HI, EOS_HI, PAD_HI]])
        pred_text = translate_sentence(src_text, model, sp_en, sp_hi)

        refs.append(tgt_text)
        hyps.append(pred_text)

# sacrebleu expects a list of references (list of list)
bleu = sacrebleu.corpus_bleu(hyps, [refs])
print(f"\n🌍 BLEU Score on Test Set: {bleu.score:.2f}\n")

# =====================
# Qualitative Examples
# =====================
sample_indices = random.sample(range(len(test_data)), 5)
print("✨ Sample Translations:")
for idx in sample_indices:
    src_text = test_data[idx]["src"]
    ref_text = test_data[idx]["tgt"]
    pred_text = translate_sentence(src_text, model, sp_en, sp_hi)
    print(f"\nEN: {src_text}")
    print(f"HI (Ref): {ref_text}")
    print(f"HI (Pred): {pred_text}")
    print("-" * 50)


C:\Users\Ayush\AppData\Local\Temp\ipykernel_13800\3408616981.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model_cnn.pt", map_l

🔍 Evaluating BLEU on test set...


Evaluating: 100%|█████████████████████████████████████████████████████████████| 1563/1563 [1:49:34<00:00,  4.21s/batch]
That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.



🌍 BLEU Score on Test Set: 19.00

✨ Sample Translations:

EN: lahore: The Pakistan English press has showered heap of praise on legendary Indian batsman Sachin Tendulkar in their editorials, saying the game of cricket will surely be poorer without him.
HI (Ref): लाहौर पाकिस्तान की अग्रेंजी प्रेस ने अपने संपादकीय में महान भारतीय बल्लेबाज सचिन तेंदुलकर की तारीफों के पुल बांधे है और लिखा है, ‘उनके बिना क्रिकेट खेल निश्चित रूप से दरिद्र’ हो जायेगा। हालांकि उर्दू प्रेस में उनके बारे में ज्यादा कुछ नहीं लिखा गया है लेकिन अंग्रेजी के अखबारों ने तेंदुलकर... आगे पढ़े
HI (Pred): पाकिस्तान के इंग्लिश प्रीमियर लीग (एआईसीसी) के प्रख्यात बल्लेबाज सचिन तेंदुलकर ने अपने संपादकीय में कहा है कि क्रिकेट का खेल निश्चित रूप से उनके बिना सबसे बड़ा होगा।
--------------------------------------------------

EN: Telecom operator Reliance Jio has announced a new plan for its prepaid customers.
HI (Ref): टेलीकॉम इंडस्ट्री में तहलका मचने वाली कंपनी रिलायंस जियो ने फिर से अपने ग्राहकों के लिए नए प्लान्स पेश किये
HI 

In [ ]:
# ============================================================
# Full Evaluation: Baseline Transformer + Hybrid CNN + BLEU
# ============================================================

import torch
import torch.nn as nn
import sentencepiece as spm
import sacrebleu
from pathlib import Path
from tqdm import tqdm

# =====================
# Config
# =====================
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MAX_LEN = 64  # pos_enc length
VOCAB_SIZE = 32000  # adjust to your SP vocab
PAD_EN = PAD_HI = 0
BOS_EN = BOS_HI = 1
EOS_EN = EOS_HI = 2

BASELINE_PATH = "checkpoint_baseline.pt"
HYBRID_PATH = "checkpoint_hybrid_cnn.pt"

SP_EN_MODEL = Path("spm_en.model")
SP_HI_MODEL = Path("spm_hi.model")

# =====================
# Load SentencePiece
# =====================
sp_en = spm.SentencePieceProcessor()
sp_hi = spm.SentencePieceProcessor()
sp_en.load(str(SP_EN_MODEL))
sp_hi.load(str(SP_HI_MODEL))

# =====================
# Transformer & Hybrid Models
# =====================
class TransformerModel(nn.Module):
    def __init__(self, src_vocab, tgt_vocab, d_model=512, nhead=8, num_layers=3, dim_feedforward=1024, dropout=0.1):
        super().__init__()
        self.src_emb = nn.Embedding(src_vocab, d_model, padding_idx=PAD_EN)
        self.tgt_emb = nn.Embedding(tgt_vocab, d_model, padding_idx=PAD_HI)
        self.pos_enc = nn.Parameter(torch.zeros(1, MAX_LEN, d_model))
        self.transformer = nn.Transformer(d_model=d_model, nhead=nhead,
                                          num_encoder_layers=num_layers,
                                          num_decoder_layers=num_layers,
                                          dim_feedforward=dim_feedforward,
                                          dropout=dropout,
                                          batch_first=True)
        self.fc_out = nn.Linear(d_model, tgt_vocab)

    def encode(self, src, src_key_padding_mask=None):
        src_emb = self.src_emb(src) + self.pos_enc[:, :src.size(1), :]
        return self.transformer.encoder(src_emb, src_key_padding_mask=src_key_padding_mask)

    def decode(self, tgt, memory, tgt_mask=None, memory_key_padding_mask=None, tgt_key_padding_mask=None):
        tgt_emb = self.tgt_emb(tgt) + self.pos_enc[:, :tgt.size(1), :]
        return self.transformer.decoder(tgt_emb, memory, tgt_mask=tgt_mask,
                                        memory_key_padding_mask=memory_key_padding_mask,
                                        tgt_key_padding_mask=tgt_key_padding_mask)

    def forward(self, src, tgt, src_key_padding_mask=None, tgt_key_padding_mask=None,
                memory_key_padding_mask=None, tgt_mask=None):
        memory = self.encode(src, src_key_padding_mask)
        output = self.decode(tgt, memory, tgt_mask, memory_key_padding_mask, tgt_key_padding_mask)
        return self.fc_out(output)

class CNNFeatureExtractor(nn.Module):
    def __init__(self, embed_dim):
        super().__init__()
        self.conv1 = nn.Conv1d(embed_dim, embed_dim, 5, padding=2)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(embed_dim, embed_dim, 3, padding=1)
        self.norm = nn.LayerNorm(embed_dim)

    def forward(self, x):
        residual = x
        x = x.transpose(1, 2)
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = x.transpose(1, 2)
        return self.norm(x + residual)

class HybridTransformerModel(nn.Module):
    def __init__(self, src_vocab, tgt_vocab, d_model=512, nhead=8, num_layers=3, dim_feedforward=1024, dropout=0.1):
        super().__init__()
        self.src_emb = nn.Embedding(src_vocab, d_model, padding_idx=PAD_EN)
        self.tgt_emb = nn.Embedding(tgt_vocab, d_model, padding_idx=PAD_HI)
        self.pos_enc = nn.Parameter(torch.zeros(1, MAX_LEN, d_model))
        self.cnn_encoder = CNNFeatureExtractor(d_model)
        self.transformer = nn.Transformer(d_model=d_model, nhead=nhead,
                                          num_encoder_layers=num_layers,
                                          num_decoder_layers=num_layers,
                                          dim_feedforward=dim_feedforward,
                                          dropout=dropout,
                                          batch_first=True)
        self.fc_out = nn.Linear(d_model, tgt_vocab)

    def encode(self, src, src_key_padding_mask=None):
        src_emb = self.src_emb(src) + self.pos_enc[:, :src.size(1), :]
        src_cnn = self.cnn_encoder(src_emb)
        return self.transformer.encoder(src_cnn, src_key_padding_mask=src_key_padding_mask)

    def decode(self, tgt, memory, tgt_mask=None, memory_key_padding_mask=None, tgt_key_padding_mask=None):
        tgt_emb = self.tgt_emb(tgt) + self.pos_enc[:, :tgt.size(1), :]
        return self.transformer.decoder(tgt_emb, memory, tgt_mask=tgt_mask,
                                        memory_key_padding_mask=memory_key_padding_mask,
                                        tgt_key_padding_mask=tgt_key_padding_mask)

    def forward(self, src, tgt, src_key_padding_mask=None, tgt_key_padding_mask=None,
                memory_key_padding_mask=None, tgt_mask=None):
        memory = self.encode(src, src_key_padding_mask)
        output = self.decode(tgt, memory, tgt_mask, memory_key_padding_mask, tgt_key_padding_mask)
        return self.fc_out(output)

# =====================
# Load checkpoints
# =====================
baseline_model = TransformerModel(VOCAB_SIZE, VOCAB_SIZE).to(DEVICE)
hybrid_model = HybridTransformerModel(VOCAB_SIZE, VOCAB_SIZE).to(DEVICE)

def load_checkpoint(model, path):
    checkpoint = torch.load(path, map_location=DEVICE)
    state_dict = checkpoint.get("model_state_dict", checkpoint)
    # Adjust pos_enc size
    if "pos_enc" in state_dict and state_dict["pos_enc"].shape[1] != MAX_LEN:
        state_dict["pos_enc"] = state_dict["pos_enc"][:, :MAX_LEN, :]
    model.load_state_dict(state_dict)
    return model

baseline_model = load_checkpoint(baseline_model, BASELINE_PATH)
hybrid_model = load_checkpoint(hybrid_model, HYBRID_PATH)

baseline_model.eval()
hybrid_model.eval()

# =====================
# Greedy translation function
# =====================
@torch.no_grad()
def translate(model, sentence, sp_en, sp_hi, max_len=MAX_LEN):
    src_ids = [BOS_EN] + sp_en.encode(sentence)[:max_len-2] + [EOS_EN]
    src = torch.tensor(src_ids, device=DEVICE).unsqueeze(0)
    tgt = torch.tensor([[BOS_HI]], device=DEVICE)

    for _ in range(max_len):
        out = model(src, tgt)
        next_token = out[:, -1, :].argmax(-1).unsqueeze(1)
        tgt = torch.cat([tgt, next_token], dim=1)
        if next_token.item() == EOS_HI:
            break

    decoded = sp_hi.decode([t for t in tgt[0].tolist() if t not in [BOS_HI, EOS_HI, PAD_HI]])
    return decoded

# =====================
# 20 sample sentences (EN + reference HI)
# =====================
test_samples = new_test_samples = [
    ("The city is preparing for heavy rainfall this week.", "शहर इस सप्ताह भारी वर्षा के लिए तैयारी कर रहा है।"),
    ("Researchers developed a new method for early disease detection.", "शोधकर्ताओं ने जल्दी रोग पहचान के लिए एक नई विधि विकसित की।"),
    ("The economy is showing signs of steady growth.", "अर्थव्यवस्था स्थिर विकास के संकेत दिखा रही है।"),
    ("Students participated enthusiastically in the science fair.", "छात्रों ने विज्ञान मेला में उत्साहपूर्वक भाग लिया।"),
    ("The festival attracted tourists from all over the world.", "त्योहार ने दुनिया भर से पर्यटकों को आकर्षित किया।"),
    ("Solar energy installations are increasing rapidly in rural areas.", "ग्रामीण क्षेत्रों में सौर ऊर्जा स्थापना तेजी से बढ़ रही है।"),
    ("The new policy aims to reduce air pollution in cities.", "नई नीति का उद्देश्य शहरों में वायु प्रदूषण को कम करना है।"),
    ("Artificial intelligence can improve healthcare diagnostics.", "कृत्रिम बुद्धिमत्ता स्वास्थ्य देखभाल निदान में सुधार कर सकती है।"),
    ("The government announced relief measures for flood-affected areas.", "सरकार ने बाढ़ प्रभावित क्षेत्रों के लिए राहत उपायों की घोषणा की।"),
    ("Wildlife conservation is critical for maintaining biodiversity.", "जंगली जीवन संरक्षण जैव विविधता बनाए रखने के लिए महत्वपूर्ण है।"),
    ("The company launched a new smartphone model last week.", "कंपनी ने पिछले सप्ताह एक नया स्मार्टफोन मॉडल लॉन्च किया।"),
    ("International trade agreements influence domestic markets.", "अंतरराष्ट्रीय व्यापार समझौते घरेलू बाजारों को प्रभावित करते हैं।"),
    ("The team developed an innovative software solution.", "टीम ने एक नवोन्मेषी सॉफ्टवेयर समाधान विकसित किया।"),
    ("Urban transport systems are facing challenges due to population growth.", "जनसंख्या वृद्धि के कारण शहरी परिवहन प्रणाली चुनौतियों का सामना कर रही है।"),
    ("The government is investing in renewable energy projects.", "सरकार नवीकरणीय ऊर्जा परियोजनाओं में निवेश कर रही है।"),
    ("Students are encouraged to engage in extracurricular activities.", "छात्रों को पाठ्येतर गतिविधियों में भाग लेने के लिए प्रोत्साहित किया जाता है।"),
    ("The company reported a decline in operating costs this quarter.", "कंपनी ने इस तिमाही में संचालन लागत में कमी की रिपोर्ट दी।"),
    ("Climate change poses a threat to coastal communities.", "जलवायु परिवर्तन तटीय समुदायों के लिए खतरा उत्पन्न करता है।"),
    ("The research team published their findings in a leading journal.", "अनुसंधान टीम ने अपने निष्कर्ष एक प्रमुख जर्नल में प्रकाशित किए।"),
    ("Global cooperation is necessary to tackle pandemics.", "महामारी से निपटने के लिए वैश्विक सहयोग आवश्यक है।"),
    ("India launched its first indigenous satellite.", "भारत ने अपना पहला स्वदेशी उपग्रह लॉन्च किया।"),
    ("The prime minister met foreign delegates at the summit.", "प्रधानमंत्री ने शिखर सम्मेलन में विदेशी प्रतिनिधियों से मुलाकात की।"),
    ("This research focuses on low-resource machine translation.", "यह शोध कम संसाधन वाली मशीन अनुवाद पर केंद्रित है।"),
    ("Its movement was captured on CCTV.", "इसके आंदोलन को सीसीटीवी में कैद किया गया।"),
    ("The two leaders discussed bilateral ties and cooperation in counter-terrorism.", "दो नेताओं ने द्विपक्षीय संबंधों और आतंकवाद निरोधक सहयोग पर चर्चा की।"),
    ("The data shows significant improvement in translation accuracy.", "डेटा अनुवाद की सटीकता में महत्वपूर्ण सुधार दिखाता है।"),
    ("He emphasized the importance of sustainable development.", "उन्होंने सतत विकास के महत्व पर जोर दिया।"),
    ("The company reported record profits this quarter.", "कंपनी ने इस तिमाही में रिकॉर्ड लाभ की सूचना दी।"),
    ("Students are encouraged to participate in research projects.", "छात्रों को अनुसंधान परियोजनाओं में भाग लेने के लिए प्रोत्साहित किया जाता है।"),
    ("The new model achieved higher BLEU scores than the baseline.", "नया मॉडल बेसलाइन की तुलना में उच्च BLEU स्कोर प्राप्त करता है।"),
    ("Climate change is affecting global agriculture.", "जलवायु परिवर्तन वैश्विक कृषि को प्रभावित कर रहा है।"),
    ("Vaccination campaigns have reduced disease incidence.", "टीकाकरण अभियानों ने रोग की घटनाओं को कम किया है।"),
    ("Artificial intelligence is transforming industries rapidly.", "कृत्रिम बुद्धिमत्ता उद्योगों को तेजी से बदल रही है।"),
    ("The movie received critical acclaim.", "इस फिल्म ने आलोचनात्मक प्रशंसा प्राप्त की।"),
    ("Electric vehicles are becoming more popular worldwide.", "विद्युत वाहन दुनिया भर में अधिक लोकप्रिय हो रहे हैं।"),
    ("Renewable energy sources are crucial for sustainability.", "नवीकरणीय ऊर्जा स्रोत सततता के लिए महत्वपूर्ण हैं।"),
    ("The government announced new education policies.", "सरकार ने नई शिक्षा नीतियों की घोषणा की।"),
    ("Space exploration has advanced significantly in recent years.", "अंतरिक्ष अन्वेषण ने हाल के वर्षों में महत्वपूर्ण प्रगति की है।"),
    ("The sports team won the national championship.", "खेल टीम ने राष्ट्रीय चैंपियनशिप जीती।"),
    ("Global trade agreements impact economic growth.", "वैश्विक व्यापार समझौते आर्थिक वृद्धि को प्रभावित करते हैं।"),
]


# =====================
# Translate & save results
# =====================
results = []
for en, hi_ref in test_samples:
    baseline_out = translate(baseline_model, en, sp_en, sp_hi)
    hybrid_out = translate(hybrid_model, en, sp_en, sp_hi)

    print(f"\nEN: {en}")
    print(f"Reference HI: {hi_ref}")
    print(f"Baseline: {baseline_out}")
    print(f"Hybrid CNN: {hybrid_out}")

    results.append({
        "EN": en,
        "Reference": hi_ref,
        "Baseline": baseline_out,
        "2 layer CNN": hybrid_out
    })

# Save results to file
with open("translation_comparison.txt", "w", encoding="utf-8") as f:
    for r in results:
        f.write(f"EN: {r['EN']}\nReference HI: {r['Reference']}\nBaseline: {r['Baseline']}\nHybrid CNN: {r['Hybrid']}\n\n")

print("\n✅ Translations saved to 'translation_comparison.txt'")

# =====================
# Compute BLEU for Hybrid model
# =====================
refs = [[r["Reference"]] for r in results]  # list of list
hyps = [r["Hybrid"] for r in results]

bleu = sacrebleu.corpus_bleu(hyps, refs)
print(f"\n🌍 BLEU Score (Hybrid CNN vs Reference): {bleu.score:.2f}")


C:\Users\Ayush\AppData\Local\Temp\ipykernel_17296\2967338906.py:125: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path, map_location=DEVICE)



EN: The city is preparing for heavy rainfall this week.
Reference HI: शहर इस सप्ताह भारी वर्षा के लिए तैयारी कर रहा है।
Baseline: इस सप्ताह तेज बारिश की तैयारियां चल रही हैं।
Hybrid CNN: इस सप्ताह तेज बारिश की तैयारी कर रहा है।

EN: Researchers developed a new method for early disease detection.
Reference HI: शोधकर्ताओं ने जल्दी रोग पहचान के लिए एक नई विधि विकसित की।
Baseline: """ ""ऑक्सफोर्डेबल्स ने शुरुआती बीमारी का पता लगाने की एक नया तरीका विकसित किया है."""
Hybrid CNN: राफैले ने जल्दी की पहचान के लिए एक नई तकनीक विकसित की.

EN: The economy is showing signs of steady growth.
Reference HI: अर्थव्यवस्था स्थिर विकास के संकेत दिखा रही है।
Baseline: अर्थव्यवस्था में स्थिरता लौटने के संकेत
Hybrid CNN: स्थिर वृद्धि के संकेत दे रहे हैं। अर्थव्यवस्था निरंतर विकास के रूप में है।, वह हमारी अर्थव्यवस्था, विश्वसनीय विकास, विश्वसनीय, विश्वसनीय विकास के, विश्वसनीय, विश्वसनीय अर्थव्यवस्था, टिकाऊ विकास के, विश्वसनीय, विश्वसनीय विकास के रूप से, विश्वसनीय, विश्वसनीय अर्थव्यवस्था, विश्वसनीय, विकास के

In [1]:
import sys
print(sys.executable)


C:\Users\Ayush\.conda\envs\mt_env\python.exe


In [2]:
# =====================
# Install & Imports
# =====================
!pip install -q datasets sentencepiece sacrebleu torch torchvision torchaudio tqdm

import os, random
from pathlib import Path
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from datasets import load_dataset
import sentencepiece as spm
import sacrebleu

# =====================
# Config
# =====================
SEED = 42
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
VOCAB_SIZE = 16000
BATCH_SIZE = 64
MAX_LEN = 64
MAX_GEN_LEN = 64
EPOCHS = 20
CLIP = 1.0
LEARNING_RATE = 3e-4
BEAM_SIZE = 5
BEST_MODEL_PATH = Path("best_model_multiscale.pt")
CHECKPOINT_PATH = Path("checkpoint_multiscale.pt")

PATIENCE = 5

torch.manual_seed(SEED)
random.seed(SEED)

# =====================
# Load dataset
# =====================
full_dataset = load_dataset("ai4bharat/samanantar", "hi", split="train")
full_dataset = full_dataset.shuffle(seed=SEED)

NUM_EXAMPLES = min(1_000_000, len(full_dataset))
subset = full_dataset.select(range(NUM_EXAMPLES))

# Split 80/10/10
train_end = int(0.8 * len(subset))
val_end = int(0.9 * len(subset))
train_data = subset.select(range(0, train_end))
val_data = subset.select(range(train_end, val_end))
test_data = subset.select(range(val_end, len(subset)))
print("Dataset sizes:", len(train_data), len(val_data), len(test_data))

# =====================
# SentencePiece
# =====================
SP_EN_MODEL = Path("spm_en.model")
SP_HI_MODEL = Path("spm_hi.model")

def write_lines(dataset_split, src_path, tgt_path):
    with open(src_path, "w", encoding="utf-8") as sf, open(tgt_path, "w", encoding="utf-8") as tf:
        for ex in dataset_split:
            sf.write(ex["src"].strip().lower() + "\n")
            tf.write(ex["tgt"].strip() + "\n")

write_lines(train_data, "train.en", "train.hi")

if not SP_EN_MODEL.exists() or not SP_HI_MODEL.exists():
    print("Training SentencePiece...")
    spm.SentencePieceTrainer.Train(
        f"--input=train.en --model_prefix=spm_en --vocab_size={VOCAB_SIZE} "
        f"--character_coverage=1.0 --model_type=unigram"
    )
    spm.SentencePieceTrainer.Train(
        f"--input=train.hi --model_prefix=spm_hi --vocab_size={VOCAB_SIZE} "
        f"--character_coverage=0.9995 --model_type=unigram"
    )

sp_en = spm.SentencePieceProcessor()
sp_hi = spm.SentencePieceProcessor()
sp_en.load(str(SP_EN_MODEL))
sp_hi.load(str(SP_HI_MODEL))

PAD_EN, BOS_EN, EOS_EN = 0, 1, 2
PAD_HI, BOS_HI, EOS_HI = 0, 1, 2

# =====================
# Dataset & DataLoader
# =====================
class NMTDataset(Dataset):
    def __init__(self, dataset, src_sp, tgt_sp, max_len=MAX_LEN):
        self.dataset = dataset
        self.src_sp = src_sp
        self.tgt_sp = tgt_sp
        self.max_len = max_len

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        src_text = self.dataset[idx]["src"].lower()
        tgt_text = self.dataset[idx]["tgt"]
        src_ids = [BOS_EN] + self.src_sp.encode(src_text)[:self.max_len-2] + [EOS_EN]
        tgt_ids = [BOS_HI] + self.tgt_sp.encode(tgt_text)[:self.max_len-2] + [EOS_HI]
        src_ids += [PAD_EN] * (self.max_len - len(src_ids))
        tgt_ids += [PAD_HI] * (self.max_len - len(tgt_ids))
        return torch.tensor(src_ids), torch.tensor(tgt_ids)

def get_loader(dataset_split, shuffle=True):
    return DataLoader(NMTDataset(dataset_split, sp_en, sp_hi),
                      batch_size=BATCH_SIZE, shuffle=shuffle)

train_loader = get_loader(train_data)
val_loader = get_loader(val_data)
test_loader = get_loader(test_data, shuffle=False)

# =====================
# Masks
# =====================
def create_padding_mask(seq, lang='en'):
    pad_id = PAD_EN if lang == 'en' else PAD_HI
    return (seq == pad_id)

def generate_square_subsequent_mask(sz):
    return torch.triu(torch.ones(sz, sz, device=DEVICE) * float('-inf'), diagonal=1)

# =====================
# Multi-Scale CNN
# =====================
class MultiScaleCNN(nn.Module):
    def __init__(self, embed_dim):
        super().__init__()
        self.conv3 = nn.Conv1d(embed_dim, embed_dim, kernel_size=3, padding=1)
        self.conv5 = nn.Conv1d(embed_dim, embed_dim, kernel_size=5, padding=2)
        self.conv7 = nn.Conv1d(embed_dim, embed_dim, kernel_size=7, padding=3)
        self.relu = nn.ReLU()
        self.norm = nn.LayerNorm(embed_dim)

    def forward(self, x):
        residual = x
        x = x.transpose(1, 2)
        out3 = self.conv3(x)
        out5 = self.conv5(x)
        out7 = self.conv7(x)
        x = out3 + out5 + out7
        x = self.relu(x)
        x = x.transpose(1, 2)
        return self.norm(x + residual)

# =====================
# Hybrid Transformer + MultiScale CNN
# =====================
class HybridTransformerModel(nn.Module):
    def __init__(self, src_vocab, tgt_vocab, d_model=512, nhead=8,
                 num_layers=3, dim_feedforward=1024, dropout=0.1, temperature=0.1):
        super().__init__()
        self.src_emb = nn.Embedding(src_vocab, d_model, padding_idx=PAD_EN)
        self.tgt_emb = nn.Embedding(tgt_vocab, d_model, padding_idx=PAD_HI)
        self.pos_enc = nn.Parameter(torch.zeros(1, MAX_LEN, d_model))
        self.cnn_encoder = MultiScaleCNN(d_model)
        self.transformer = nn.Transformer(
            d_model=d_model, nhead=nhead,
            num_encoder_layers=num_layers, num_decoder_layers=num_layers,
            dim_feedforward=dim_feedforward, dropout=dropout, batch_first=True
        )
        self.fc_out = nn.Linear(d_model, tgt_vocab)
        self.temperature = temperature

    def encode(self, src, src_key_padding_mask):
        src_emb = self.src_emb(src) + self.pos_enc[:, :src.size(1), :]
        src_cnn = self.cnn_encoder(src_emb)
        return self.transformer.encoder(src_cnn, src_key_padding_mask=src_key_padding_mask)

    def decode(self, tgt, memory, tgt_mask, memory_key_padding_mask, tgt_key_padding_mask):
        tgt_emb = self.tgt_emb(tgt) + self.pos_enc[:, :tgt.size(1), :]
        return self.transformer.decoder(
            tgt_emb, memory,
            tgt_mask=tgt_mask,
            memory_key_padding_mask=memory_key_padding_mask,
            tgt_key_padding_mask=tgt_key_padding_mask
        )

    def forward(self, src, tgt, src_key_padding_mask=None, tgt_key_padding_mask=None,
                memory_key_padding_mask=None, tgt_mask=None):
        memory = self.encode(src, src_key_padding_mask)
        output = self.decode(tgt, memory, tgt_mask, memory_key_padding_mask, tgt_key_padding_mask)
        return self.fc_out(output)

    # -----------------------
    # Contrastive loss
    # -----------------------
    def contrastive_loss(self, anchor, positive, negative):
        anchor = anchor.mean(dim=1)
        positive = positive.mean(dim=1)
        negative = negative.mean(dim=1)
        pos_sim = torch.cosine_similarity(anchor, positive, dim=-1)
        neg_sim = torch.cosine_similarity(anchor, negative, dim=-1)
        loss = -torch.log(torch.exp(pos_sim / self.temperature) / (torch.exp(pos_sim / self.temperature) + torch.exp(neg_sim / self.temperature)))
        return loss.mean()

# =====================
# Initialize model
# =====================
model = HybridTransformerModel(len(sp_en), len(sp_hi)).to(DEVICE)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_HI)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=LEARNING_RATE, steps_per_epoch=len(train_loader), epochs=EPOCHS
)
scaler = torch.amp.GradScaler()

start_epoch = 1
best_val_loss = float("inf")
epochs_no_improve = 0

# =====================
# Resume checkpoint
# =====================
if CHECKPOINT_PATH.exists():
    print(f"Resuming from {CHECKPOINT_PATH}...")
    checkpoint = torch.load(CHECKPOINT_PATH, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    scaler.load_state_dict(checkpoint['scaler_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    best_val_loss = checkpoint['best_val_loss']
    epochs_no_improve = checkpoint['epochs_no_improve']
    print(f"Checkpoint loaded — Resuming from epoch {start_epoch}")
else:
    print("Starting training from scratch")

# =====================
# Training Loop
# =====================
for epoch in range(start_epoch, EPOCHS + 1):
    model.train()
    train_loss = 0
    contrastive_loss_total = 0

    for src, tgt in tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}", unit="batch"):
        src, tgt = src.to(DEVICE), tgt.to(DEVICE)
        src_mask = create_padding_mask(src, 'en')
        tgt_input, tgt_out = tgt[:, :-1], tgt[:, 1:]
        tgt_mask = generate_square_subsequent_mask(tgt_input.size(1))
        tgt_key_padding_mask = create_padding_mask(tgt_input, 'hi')

        optimizer.zero_grad()
        with torch.amp.autocast(device_type="cuda"):
            logits = model(
                src, tgt_input,
                src_key_padding_mask=src_mask,
                tgt_key_padding_mask=tgt_key_padding_mask,
                memory_key_padding_mask=src_mask,
                tgt_mask=tgt_mask
            )
            ce_loss = criterion(logits.reshape(-1, logits.size(-1)), tgt_out.reshape(-1))

            # -----------------------------
            # Contrastive loss
            # -----------------------------
            neg_tgt_input = tgt_input[torch.randperm(tgt_input.size(0))].to(DEVICE)
            memory = model.encode(src, src_mask)
            pos_repr = model.decode(tgt_input, memory, tgt_mask, src_mask, tgt_key_padding_mask)
            neg_repr = model.decode(neg_tgt_input, memory, tgt_mask, src_mask, tgt_key_padding_mask)
            cl_loss = model.contrastive_loss(pos_repr, pos_repr, neg_repr)

            loss = ce_loss + 0.1 * cl_loss
            contrastive_loss_total += cl_loss.item()

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP)
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        train_loss += ce_loss.item()

    avg_train_loss = train_loss / len(train_loader)
    avg_cl_loss = contrastive_loss_total / len(train_loader)
    print(f"Epoch {epoch} | CE Loss: {avg_train_loss:.4f} | Contrastive Loss: {avg_cl_loss:.4f}")

    # ---- Validation ----
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for src, tgt in val_loader:
            src, tgt = src.to(DEVICE), tgt.to(DEVICE)
            src_mask = create_padding_mask(src, 'en')
            tgt_input, tgt_out = tgt[:, :-1], tgt[:, 1:]
            tgt_mask = generate_square_subsequent_mask(tgt_input.size(1))
            tgt_key_padding_mask = create_padding_mask(tgt_input, 'hi')

            logits = model(
                src, tgt_input,
                src_key_padding_mask=src_mask,
                tgt_key_padding_mask=tgt_key_padding_mask,
                memory_key_padding_mask=src_mask,
                tgt_mask=tgt_mask
            )
            loss = criterion(logits.reshape(-1, logits.size(-1)), tgt_out.reshape(-1))
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"Epoch {epoch} | Val Loss: {avg_val_loss:.4f}")

    # ---- Save Checkpoint & Early Stopping ----
    torch.save({
        "epoch": epoch,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "scheduler_state_dict": scheduler.state_dict(),
        "scaler_state_dict": scaler.state_dict(),
        "best_val_loss": best_val_loss,
        "epochs_no_improve": epochs_no_improve,
    }, CHECKPOINT_PATH)

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        print("New best model saved!")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= PATIENCE:
            print("Early stopping triggered.")
            break

print("Training complete ✅")


C:\Users\Ayush\.conda\envs\mt_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset sizes: 800000 100000 100000
Resuming from checkpoint_multiscale.pt...


C:\Users\Ayush\AppData\Local\Temp\ipykernel_4408\4167527191.py:222: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(CHECKPOINT_PATH, map_location=DEVIC

Checkpoint loaded — Resuming from epoch 22
Training complete ✅
